In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-155289
Azure region: southcentralus
Subscription id: d4ad7261-832d-46b2-b093-22156001df5b
Resource group: aml-quickstarts-155289


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException
cluster_name = "azuremlCluster"
try:
    compute_cluster = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

InProgress......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
from azureml.core import Environment

# Specify an sklearn - enabled environment
sklearn_env = Environment.get(workspace=ws, name="AzureML-AutoML")

# Specify parameter sampler
ps = RandomParameterSampling(
    {'C': uniform(0.001, 4.0),
    'max_iter': choice([1000, 1500, 2000, 2500, 5000])}
)
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1, delay_evaluation=5)
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters
if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
#est = SKLearn(source_directory=".", entry_script='train.py', compute_target=compute_cluster)
est = ScriptRunConfig(source_directory='.', script='train.py', 
    compute_target=compute_cluster,
    environment=sklearn_env   
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=est,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10
)

In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
experiment = Experiment(ws, "HyperdriveExperiment")
hyperdrive_run = experiment.submit(config=hyperdrive_config)
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_9581cde8-b3a1-4144-b4aa-416c419f9531
Web View: https://ml.azure.com/runs/HD_9581cde8-b3a1-4144-b4aa-416c419f9531?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-155289/workspaces/quick-starts-ws-155289&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-21T15:39:43.088173][API][INFO]Experiment created<END>\n""<START>[2021-08-21T15:39:43.526863][GENERATOR][INFO]Trying to sample '10' jobs from the hyperparameter space<END>\n""<START>[2021-08-21T15:39:43.742833][GENERATOR][INFO]Successfully sampled '10' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_9581cde8-b3a1-4144-b4aa-416c419f9531
Web View: https://ml.azure.com/runs/HD_9581cde8-b3a1-4144-b4aa-416c419f9531?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-155289/workspaces/quick-starts-ws-155289&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_9581cde8-b3a1-4144-b4aa-416c419f9531',
 'target': 'azuremlCluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-21T15:39:42.827131Z',
 'endTimeUtc': '2021-08-21T15:49:47.785714Z',
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'b062a630-601d-418a-b164-c2d85f63e3b2',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1055-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.33.0',
  'score': '0.912451719698363',
  'best_child_run_id': 'HD_9581cde8-b3a1-4144-b4aa-416c419f9531_5',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg155289.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_9581cde8-b3a1-4144-b4aa-416c419f9531/azureml-logs/hyperdrive.txt?sv=2019-07-07&sr=b

In [5]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
from azureml.widgets import RunDetails
best_run = hyperdrive_run.get_best_run_by_primary_metric()
RunDetails(hyperdrive_run).show()

if "outputs" not in os.listdir():
    os.mkdir("./outputs")
best_run.download_file('outputs/model.joblib', output_file_path='outputs/best_hyperdrive_model.joblib')
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.run(class)?view=azure-ml-py
# This took me so much time to figure out

#best_run_metrics = best_run.get_metrics()
#parameter_values = best_run.get_details()['runDefinition']['arguments']

#print('Best Run Id: ', best_run.id)
#print('\n Accuracy:', best_run_metrics['accuracy'])




_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [7]:
from train import clean_data
from azureml.core.dataset import Dataset
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
x_concat = pd.concat([x, y], axis=1)

# Register x dataframe as a dataset to be consumed by automl
datastore = ws.get_default_datastore()
TabularDatasetFactory.register_pandas_dataframe(
    dataframe=x_concat,
    target=datastore,
    name="x_tabular_dataset"
)
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.dataset_factory.tabulardatasetfactory?view=azure-ml-py#register-pandas-dataframe-dataframe--target--name--description-none--tags-none--show-progress-true-
x_tabular_dataset = Dataset.get_by_name(name='x_tabular_dataset',
    workspace=ws)

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/27a798a2-c86b-45ed-bbda-f070bc5dc9f6/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


In [8]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30, # To not allow experiments to run indefinitely. 
    task="classification", # To establish the type of algorithms to apply. We are interested in classifying customers as likely to buy
    primary_metric="accuracy", # Accuracy was a choice, I could have also chosen AUC. Accuracy maximizes a particular model's predictive power, but may not be as generally robust as if we had optimized for AUC
    training_data=x_tabular_dataset,
    label_column_name="y",
    n_cross_validations=5, # Instructions to ensure that the data is segmented 5 times for validation purposes. This process helps ensure the model is more robust, and still fully utilizes all the data for training
    compute_target=compute_cluster
)

In [9]:
# Submit your automl run

### YOUR CODE HERE ###
automl_experiment = Experiment(ws, 'automl_experiment')
run = automl_experiment.submit(config=automl_config)
run.wait_for_completion(show_output=True)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_57639393-41c4-4bd6-8e1f-1f51a1295b62,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


Experiment,Id,Type,Status,Details Page,Docs Page
automl_experiment,AutoML_57639393-41c4-4bd6-8e1f-1f51a1295b62,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the sm

{'runId': 'AutoML_57639393-41c4-4bd6-8e1f-1f51a1295b62',
 'target': 'azuremlCluster',
 'status': 'Completed',
 'startTimeUtc': '2021-08-21T15:51:26.883492Z',
 'endTimeUtc': '2021-08-21T16:35:12.2284Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'azuremlCluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_experiment","subscription_id":"d4ad7261-832d-46b2-b093-22156001df5b","resource_group":"aml-quickstarts-155289","workspace_name":"quick-starts-ws-155289","region":"southcentralus","compute_target":"azuremlCluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"a

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, model = run.get_output()
joblib.dump(model, 'best_automl_model.joblib')
compute_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

